In [1]:
from sklearn import svm, metrics
import glob, os.path, re, json

# re: 정규표현식
# 파일 저장 시 json 형태로 저장하기 위해 json import

#### Code 해석 순서
1. main 부터 찾아서 해석
2. 함수 선언은 main 위쪽 ( 호출이 빠른 순서대로 main과 가까운 위치 )

In [2]:
def check_freq(fname):
#     file open + 음절계산
    
    with open(fname, 'r', encoding='utf-8') as f:
#         readLines() 쓰지 않은 이유: 음절 단위로 끊으므로 줄바꿈 고려X
        text = f.read()        
        pass
    text = text.lower()
    
    cnt = [ 0 for n in range(0,26) ]
#     메모리 비효율성( 문자 -> 숫자 -> 문자 ) 해결 위해 Ascii 코드 변환
    code_a = ord('a')
    code_z = ord('z')
    
    for ch in text:
        n = ord(ch)
        if code_a <= n <= code_z:
            cnt[ n - code_a ] += 1
            pass
        pass
    
#     정규화하기 ( text간 상대 비교하기 위해 )
    total = sum(cnt)
    freq = list( map( lambda x: x / total, cnt ) )
    
#     lang????
    name = os.path.basename(fname)
#     r: unicode 방식에 맞춰 매핑(즉, 한글도 포함)
#     ^: 시작점부터 // group(): match 결과 반환
    lang = re.match(r'^[a-z]{2,}', name).group()
        
    return freq, lang
    


def load_files(path):
    freqs = []
    labels = []
    file_list = glob.glob(path)
    for fname in file_list:
        r = check_freq(fname)
        freqs.append(r[0])
        labels.append(r[1])
        pass
    # 전처리된 dataset을 json 형태로 저장하기 위한 data 구조
    return {'freqs': freqs, 'labels': labels}


##### main 시작

data = load_files('./lang/train/*.txt')
test = load_files('./lang/test/*.txt')

# 전처리된 dataset을 json 형태로 저장
with open('./lang/freq.json', 'w', encoding = 'utf-8') as fp:
    json.dump([data, test],fp)
    pass

### 모델 학습

SVM(Support Vector Machine):<br>경계선에 놓여있는 데이터들을 분류/수치화 하는 알고리즘<br>SVC: 분류

In [17]:
# json 파일 읽기
# import json

# with open('./lang/freq.json', 'r', encoding='utf-8') as fp:
#     data = json.load(fp)

# gamma=20 줄 경우 정답률 1 --> overfitting??
clf = svm.SVC()
clf.fit(data['freqs'], data['labels'])

C:\Users\student\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

### 예측

In [18]:
predict = clf.predict(test['freqs'])

### 결과 테스트

In [19]:
ac_score = metrics.accuracy_score(test['labels'], predict)
cl_report = metrics.classification_report(test['labels'], predict)
print('정답률 = ', ac_score)
print('리포트 =')
print(cl_report)

정답률 =  0.875
리포트 =
              precision    recall  f1-score   support

          en       0.67      1.00      0.80         2
          fr       1.00      1.00      1.00         2
          id       1.00      0.50      0.67         2
          tl       1.00      1.00      1.00         2

    accuracy                           0.88         8
   macro avg       0.92      0.88      0.87         8
weighted avg       0.92      0.88      0.87         8



예측률
Accuracy = ( TP + TF ) / ( TP + FP + FN + TF )

예측치 기준 정확도
Precision = TP / (TP + FP)

실측치 기준 재현율
recall = TP / ( TP + FN )

ex) id해석: 예측한 경우는 다 맞았으나 실제 데이터와 비교 시 반만 맞춤

f1-score = 2 * 1 / ( 1/Precision + 1/Recall )<br> = 2 * Precision * Recall / ( Precision + Recall )<br>데이터 label이 불균형 구조일 때 모델의 성능을 정확하게 평가하는 지표( 단순 산술평균을 사용하지 않는 이유 )

### 학습 모델 저장

In [21]:
import json, joblib
# 각 언어의 출현 빈도 데이터 읽어 들이기
with open('./lang/freq.json', 'r', encoding='utf-8') as fp:
    d = json.load(fp)
    data = d[0]

# 데이터 학습하기
clf = svm.SVC()
clf.fit(data['freqs'], data['labels'])
#학습 데이터 저장하기
joblib.dump(clf, './lang/freq.pkl')
print('ok')

ok


C:\Users\student\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


### 웹 서버 생성 ( AI server - Web server )

#### 이하의 코드는 lang-Webapp.py 로 저장

server에서는 무조건 python 코드를 먼저 해석하고 이후 결과값을 html 텍스트 형식으로 리턴